# Загрузка Pandas и очистка данных

In [32]:
import pandas as pd

import re
import random

In [2]:
df = pd.read_csv(r'proj3_main_task.csv')

In [3]:
city = pd.read_excel(r'cities_unique.xlsx')

In [4]:
cuisine = df['Cuisine Style']

cuisine = cuisine.dropna()
cuisine = cuisine.str.strip("[]").str.split(', ')
cuisine = cuisine.explode()
cuisine = cuisine.str.strip("''")
cuisine.index = [x for x in range(len(cuisine))]

In [5]:
csn_sorted = cuisine.value_counts().index.to_list()

In [6]:
csn_top = csn_sorted[:20]

In [7]:
# replacing NaN in 'Cuisine Style'
df['Cuisine Style'] = df['Cuisine Style'].fillna('no data')

In [8]:
# gathering list for trending cuisines in DataFrame

l = []

for i in range(len(df['Cuisine Style'])):
    l1 = []
    for j in csn_top:
        if j in df['Cuisine Style'][i]:
            l1.append(1)
        else:
            l1.append(0)
    if 1 in l1:
        l.append(1)
    else:
        l.append(0)

In [9]:
# new parameter - 'trending' - trending cuisine
df['Trending'] = l

In [10]:
df = df.merge(city, how = 'left')

In [11]:
df.head(5)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,Trending,Population,Population density,Capital,avg_salary,Area,intl_visitors_mln,world_best_city rating_pos,cuisine
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,1,2.176,20641,1,3147.00,105.40,17.560,3,'French'
1,id_1535,Stockholm,no data,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,0,0.976,5200,1,3269.17,188.00,2.604,59,'Swedish'
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,1,8.962,5666,1,3882.00,1572.00,19.233,1,'British'
3,id_3456,Berlin,no data,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,0,3.770,4228,1,2937.38,891.70,5.959,18,'German'
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,1,1.484,4800,0,3666.61,310.43,4.066,28,'German'


In [12]:
# new parameter 'local' - local cuisine

df['local'] = [1 if df['cuisine'][x] in df['Cuisine Style'][x] else 0 for x in range(len(df))]

In [14]:
df['Number_of_Reviews_isNAN'] = pd.isna(df['Number of Reviews']).astype('uint8')
df['Cuisine Style_isNAN'] = pd.isna(df['Cuisine Style']).astype('uint8')
df['Price Range_isNAN'] = pd.isna(df['Number of Reviews']).astype('uint8')

In [15]:
df['Cuisine Style'] = df['Cuisine Style'].str.strip('[]').str.split(', ')
df['Cuisine Style'] = df['Cuisine Style'].fillna(0)

def lenlst(x):
    if x == 0:
        return 1
    else:
        return len(x)
    
csn_cnt = df['Cuisine Style'].apply(lambda x: lenlst(x))

# new parameter - number of cuisines in restaurant
df['Cuisine count'] = csn_cnt

In [16]:
df1 = df.copy()

pattern = re.compile('\d+\W\d+\W\d\d\d\d')
df1['rev_dates'] = df1['Reviews'].apply(lambda x: pattern.findall(str(x)))

delta = []

for i in df1['rev_dates']:
    if len(i) < 2:
        delta.append(0)
    else:
        delta.append((pd.to_datetime(i[0]) - pd.to_datetime(i[1])).days)

# new parameter - days between feedbacks
df['timedelta'] = delta
df['timedelta'] = abs(df['timedelta'])

In [17]:
df1['price_range_mod'] = df1['Price Range']
df1['price_range_mod'] = df1['price_range_mod'].replace('$', int(1))
df1['price_range_mod'] = df1['price_range_mod'].replace('$$ - $$$', int(2))
df1['price_range_mod'] = df1['price_range_mod'].replace('$$$$', int(3))
df1['price_range_mod'] = df1['price_range_mod'].fillna(random.randint(1, 3))

# new parameter - price range
df['price_range_mod'] = df1['price_range_mod']

In [18]:
df.head(5)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,...,intl_visitors_mln,world_best_city rating_pos,cuisine,local,Number_of_Reviews_isNAN,Cuisine Style_isNAN,Price Range_isNAN,Cuisine count,timedelta,price_range_mod
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,...,17.560,3,'French',1,0,0,0,3,41,2.0
1,id_1535,Stockholm,[no data],1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,...,2.604,59,'Swedish',0,0,0,0,1,382,3.0
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,...,19.233,1,'British',0,0,0,0,7,2,3.0
3,id_3456,Berlin,[no data],3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,...,5.959,18,'German',0,0,0,0,1,0,3.0
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,...,4.066,28,'German',1,0,0,0,3,272,2.0


In [19]:
# dummies for 'City'

df = pd.get_dummies(df, columns=['City'], dummy_na=True)

In [20]:
# replacing 'nan' in 'Number of Reviews'

df['Number of Reviews'] = df['Number of Reviews'].fillna(0)

In [21]:
# df = df.explode('Cuisine Style')
# df = pd.get_dummies(df, columns=['Cuisine Style'], dummy_na=True)

In [22]:
df.head(5)

,Restaurant_id,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,Trending,...,City_Oporto,City_Oslo,City_Paris,City_Prague,City_Rome,City_Stockholm,City_Vienna,City_Warsaw,City_Zurich,City_nan
0,id_5569,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,1,...,0,0,1,0,0,0,0,0,0,0
1,id_1535,[no data],1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,0,...,0,0,0,0,0,1,0,0,0,0
2,id_352,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,1,...,0,0,0,0,0,0,0,0,0,0
3,id_3456,[no data],3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,0,...,0,0,0,0,0,0,0,0,0,0
4,id_615,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,1,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df = df.drop(columns=['Price Range', 'Cuisine Style', 'Reviews', 'URL_TA', 'ID_TA', 'cuisine'])

In [24]:
df.head(5)

,Restaurant_id,Ranking,Rating,Number of Reviews,Trending,Population,Population density,Capital,avg_salary,Area,...,City_Oporto,City_Oslo,City_Paris,City_Prague,City_Rome,City_Stockholm,City_Vienna,City_Warsaw,City_Zurich,City_nan
0,id_5569,5570.0,3.5,194.0,1,2.176,20641,1,3147.00,105.40,...,0,0,1,0,0,0,0,0,0,0
1,id_1535,1537.0,4.0,10.0,0,0.976,5200,1,3269.17,188.00,...,0,0,0,0,0,1,0,0,0,0
2,id_352,353.0,4.5,688.0,1,8.962,5666,1,3882.00,1572.00,...,0,0,0,0,0,0,0,0,0,0
3,id_3456,3458.0,5.0,3.0,0,3.770,4228,1,2937.38,891.70,...,0,0,0,0,0,0,0,0,0,0
4,id_615,621.0,4.0,84.0,1,1.484,4800,0,3666.61,310.43,...,0,0,0,0,0,0,0,0,0,0


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000 entries, 0 to 39999
Data columns (total 51 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Restaurant_id               40000 non-null  object 
 1   Ranking                     40000 non-null  float64
 2   Rating                      40000 non-null  float64
 3   Number of Reviews           40000 non-null  float64
 4   Trending                    40000 non-null  int64  
 5   Population                  40000 non-null  float64
 6   Population density          40000 non-null  int64  
 7   Capital                     40000 non-null  int64  
 8   avg_salary                  40000 non-null  float64
 9   Area                        40000 non-null  float64
 10  intl_visitors_mln           40000 non-null  float64
 11  world_best_city rating_pos  40000 non-null  int64  
 12  local                       40000 non-null  int64  
 13  Number_of_Reviews_isNAN     400

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [26]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [27]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [28]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [29]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [30]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [31]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.20882849999999997
